In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import os
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
data = "/content/drive/MyDrive/hoa/hoa"
images = []
labels = []
class_id = 0
label_map = {}
for image_name in os.listdir(data):
    image_path = os.path.join(data, image_name)
    image = Image.open(image_path).convert('RGB')
    image = image.resize((128, 128)) # Tăng kích thước ảnh
    image = np.array(image)

    images.append(image)
    if image_name.split('_')[0] not in label_map:
      label_map[image_name.split('_')[0]] = class_id
      class_id += 1

    label = label_map[image_name.split('_')[0]]
    labels.append(label)
images = np.array(images)
labels = np.array(labels)
print(label_map)

{'hoa hue': 0, 'hoa tulip': 1, 'hoa sen': 2, 'hoa ly': 3, 'hoa hong': 4, 'hoa cuc': 5}


In [ ]:
# Chuẩn hóa giá trị pixel
images = images.astype('float32') / 255.0

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Mã hóa nhãn bằng one-hot encoding
num_classes = len(np.unique(labels))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [ ]:
model = Sequential([
    Input(shape=(128, 128, 3)),  # ✅ Đổi input thành (128, 128, 3)

    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Dropout(0.4),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')  # ✅ Khớp với y_train có (67, 6)
])


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100,validation_data=(X_test, y_test))

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 36s 9s/step - accuracy: 0.1719 - loss: 87.6254 - val_accuracy: 0.3529 - val_loss: 2.8916
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 39s 8s/step - accuracy: 0.2135 - loss: 138.5118 - val_accuracy: 0.0588 - val_loss: 2.2655
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 40s 8s/step - accuracy: 0.2245 - loss: 56.5583 - val_accuracy: 0.1765 - val_loss: 3.0457
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 42s 9s/step - accuracy: 0.4576 - loss: 35.3041 - val_accuracy: 0.0588 - val_loss: 3.6498
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 32s 10s/step - accuracy: 0.4384 - loss: 25.6981 - val_accuracy: 0.0588 - val_loss: 3.1645
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 38s 8s/step - accuracy: 0.6298 - loss: 10.3454 - val_accuracy: 0.0588 - val_loss: 4.3967
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 43s 9s/step - accuracy: 0.5957 - loss: 10.0464 - val_accuracy: 0.0588 - val_loss: 3.2358
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 39s 8s/step - accuracy: 0.6071 - loss: 7.3189 - val_accuracy: 0.1765 - val_loss

In [ ]:
loss, accuracy = model.evaluate(X_test,y_test)
print(f'độ chính xác : {int(accuracy*100)}%')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.1765 - loss: 3.0606
độ chính xác : 17%


In [ ]:
model.save('trainflower.h5')

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

def predict_digit(image_path, model):
    # Load ảnh và resize về (128, 128)
    img = image.load_img(image_path, target_size=(128, 128))
    img_array = image.img_to_array(img)  # Chuyển thành mảng numpy
    img_array = img_array / 255.0  # Chuẩn hóa dữ liệu
    img_array = np.expand_dims(img_array, axis=0)  # Thêm batch dimension (1, 128, 128, 3)

    # Dự đoán
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)  # Lấy nhãn có xác suất cao nhất
    return predicted_class

# Load mô hình đã train
model = load_model("/content/trainflower.h5")  # Thay đường dẫn phù hợp

# Dự đoán ảnh
image_path = "/content/hoa sen_6.jpg"
predicted_digit = predict_digit(image_path, model)
print(f"Số tiên đoán: {predicted_digit}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 484ms/step
Số tiên đoán: 2


In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import os
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Đọc dữ liệu ảnh
data = "/content/drive/MyDrive/hoa/hoa"
images = []
labels = []
class_id = 0
label_map = {}

for image_name in os.listdir(data):
    image_path = os.path.join(data, image_name)
    image = Image.open(image_path).convert('RGB')
    image = image.resize((128, 128))  # Tăng kích thước ảnh
    image = np.array(image)
    images.append(image)

    if image_name.split('_')[0] not in label_map:
        label_map[image_name.split('_')[0]] = class_id
        class_id += 1

    label = label_map[image_name.split('_')[0]]
    labels.append(label)

images = np.array(images)
labels = np.array(labels)

# Chuẩn hóa giá trị pixel
images = images.astype('float32') / 255.0

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Mã hóa nhãn bằng one-hot encoding
num_classes = len(np.unique(labels))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    zoom_range=0.3,
    brightness_range=[0.5, 1.5],
    shear_range=0.2,
    fill_mode='nearest'
)
datagen.fit(X_train)

# Mô hình CNN cải tiến
model = Sequential([
    Input(shape=(128, 128, 3)),

    # Block 1
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Dropout(0.4),
    Conv2D(512, (3, 3), activation='relu', padding='same'),
    Conv2D(512, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Dropout(0.5),

    GlobalAveragePooling2D(),  # Giảm số tham số để tránh overfitting

    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile mô hình
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50,validation_data=(X_test, y_test))
loss, accuracy = model.evaluate(X_test,y_test)
print(f'độ chính xác : {int(accuracy*100)}%')

Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 56s 13s/step - accuracy: 0.1492 - loss: 1.8455 - val_accuracy: 0.0588 - val_loss: 1.7980
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 49s 14s/step - accuracy: 0.2060 - loss: 1.7895 - val_accuracy: 0.0588 - val_loss: 1.7974
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 82s 14s/step - accuracy: 0.1797 - loss: 1.7909 - val_accuracy: 0.0588 - val_loss: 1.7990
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 82s 14s/step - accuracy: 0.2444 - loss: 1.7809 - val_accuracy: 0.1176 - val_loss: 1.8015
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 82s 14s/step - accuracy: 0.1911 - loss: 1.7693 - val_accuracy: 0.0588 - val_loss: 1.7879
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 82s 15s/step - accuracy: 0.2828 - loss: 1.7356 - val_accuracy: 0.0588 - val_loss: 1.7573
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 82s 14s/step - accuracy: 0.2405 - loss: 1.6672 - val_accuracy: 0.0588 - val_loss: 2.0578
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 81s 14s/step - accuracy: 0.2021 - loss: 1.6863 - val_accuracy: 0.0588 - val_loss: 1.7135


In [ ]:
model.save('hoa.h5')

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load mô hình đã lưu
model = tf.keras.models.load_model('/content/hoa.h5')

# Define class_labels manually, since train_generator is not used
# Assuming your label_map from previous code is: {'hoa hong': 0, 'hoa cuc': 1, 'hoa ly': 2, 'hoa mai': 3, 'hoa sen': 4, 'hoa lan': 5}
class_labels = {0: 'hoa hong', 1: 'hoa cuc', 2: 'hoa ly', 3: 'hoa mai', 4: 'hoa sen', 5: 'hoa lan'}

def predict_image(img_path):
    # Load và tiền xử lý ảnh
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img) / 255.0  # Chuẩn hóa về [0, 1]
    img_array = np.expand_dims(img_array, axis=0)  # Mở rộng chiều batch

    # Dự đoán
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)  # Lấy lớp có xác suất cao nhất
    predicted_label = class_labels[predicted_class]  # Lấy tên lớp

    # In kết quả
    print(f'Ảnh: {img_path}')
    print(f'Dự đoán: {predicted_label}')

# Dùng hàm để dự đoán ảnh mới
image_path = '/content/13.jpg'  # Đường dẫn ảnh mới
# pyplot.imshow(image.load_img(image_path)) # Assuming pyplot is imported and available
predict_image(image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step
Ảnh: /content/13.jpg
Dự đoán: hoa sen
